In [53]:
import numpy as np
import pandas as pd
import xarray as xr
import math
import dask
import glob
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt

In [54]:
files = glob.glob("../FRONT/FRONT_FILES/native/seasonal/*.nc")

models = []
for file in files:
    models.append(file.split('/')[-1].split('_')[0])

In [55]:
NOAA_2deg = xr.open_dataset('../FRONT/FRONT_FILES/2deg/seasonal/NOAA_seasonal_fronts_2deg.nc')
new_lat = NOAA_2deg.lat.values
new_lon = NOAA_2deg.lon.values

In [68]:
dic = {}
for model,file in zip(models,files):
    dic[model] = xr.open_dataset(file)
    dic[model] = dic[model].rename({'__xarray_dataarray_variable__':'front'})  
    dic[model] = dic[model].front.fillna(0)
    dic[model] = dic[model].sel(time = slice('1981-03-01','2005-12-30'))  #change from 1950/1981 here
    dic[model] = dic[model].groupby(dic[model].time.dt.season).mean(dim='time',skipna=True)
    dic[model] = dic[model].interp(lat=new_lat, lon=new_lon,method='linear')  #interpolate to 2deg grid
    dic[model] = dic[model].sel(lat = slice(-60,-25))
    dic[model] = dic[model].sel(lon = slice(-30,10))
    dic[model] = dic[model]/(np.max(dic[model].values))

In [69]:
dic_native = dic

In [70]:
weights = np.cos(np.deg2rad(dic_native['NOAA'].lat)) #area weighted

In [73]:
#mean absolute error calc
scores=[]
for reference in dic_native:
    for index in dic_native:
        MAE={}
        for season in ['DJF','MAM','JJA','SON']:
            ref = dic_native[reference].sel(season = season)
            x = dic_native[index].sel(season = season)
            MAE[season] = (np.abs(ref - x)).weighted(weights).sum(('lat','lon'))
        scores.append([reference, index,np.mean(MAE['DJF'].values + MAE['MAM'].values + MAE['JJA'].values + MAE['SON'].values)])
        
models = list(dic_native.keys())

In [74]:
df = pd.DataFrame(models,columns = ['models'])
start = 0
end = len(models)
for index in models:
    df[index] = np.array(scores[start:end])[:,-1].astype(float)
    start = start + len(models)
    end = end + len(models)

In [75]:
df = df.set_index('models')

df.to_csv('Front_Linkage.csv')